In [3]:
import pandas as pd

In [9]:
PATH = '2017/Student-Feedback-Surveys-Superview.csv'

with open(PATH) as fr:
    data = fr.readlines()
    
for index, line in enumerate(data):
    data[index] = line.split(",")


In [10]:
pd.DataFrame(data)

,0,1,2,3,4,5,6
0,﻿ID,Location,Track,Week,Rating (Num),Schedule Pacing\n,None
1,134,San Francisco,"""Apps","Explorer""",Week 1,3,Just right\n
2,36,Los Angeles,Apps,Week 1,4,A little too fast\n,None
3,117,San Francisco,Games,Week 1,4,Way too slow\n,None
4,253,,,Week 2,4,A little too fast\n,None
5,350,New York City,"""Apps","Explorer""",Week 1,4,Just right\n
6,23,Redwood City,Apps,Week 1,5,Just right\n,None
7,28,Los Angeles,Apps,Week 7,5,Just right\n,None
8,65,San Francisco,Apps,Week 1,5,A little too slow\n,None
9,101,Santa Clara,Apps,Week 1,5,A little too slow\n,None
